# Cypher Query Examples

In [ ]:
#@title Check if Notebook is running in Google Colab
in_colab = False
try:
    import google.colab
    # enable third party widgets in Colab
    from google.colab import output
    output.enable_custom_widget_manager()
    in_colab = True
except:
    pass

In [ ]:
#@title Install software (Google Colab only)
if in_colab:
    !apt -qq install openjdk-17-jre-headless
    !wget -q https://raw.githubusercontent.com/pwrose/neo4j-ipycytoscape/master/notebooks/neo4j_utils.py
    !wget -q https://raw.githubusercontent.com/pwrose/kg-import/master/notebooks/neo4j_bulk_importer.py
    !wget -q https://raw.githubusercontent.com/pwrose/neo4j-ipycytoscape/master/.env.colab
    %pip install -q python-dotenv
    from dotenv import load_dotenv
    load_dotenv(".env.colab")
    %pip install py2neo --quiet
    %pip install ipycytoscape --quiet

In [ ]:
import neo4j_utils

In [ ]:
neo4j_utils.start()

In [ ]:
import neo4j_blulk_importer

In [ ]:
neo4j_blulk_importer.create_kg()

In [ ]:
neo4j_utils.stop()

In [1]:
import os
import pandas as pd
from py2neo import Graph
import neo4j_utils

In [2]:
pd.set_option('display.max_colwidth', None)

### Connect to local Neo4j Graph database

In [3]:
DEMO_SERVER_URL = os.getenv("DEMO_SERVER_URL")
DEMO_SERVER_UN = os.getenv("DEMO_SERVER_UN")
DEMO_SERVER_PW = os.getenv("DEMO_SERVER_PW")
graph = Graph(DEMO_SERVER_URL, user=DEMO_SERVER_UN, password=DEMO_SERVER_PW)

In [4]:
#graph = Graph("bolt://localhost:7687", name="neo4j", user="neo4j", password="neo4jdemo")

### Display Node metadata

In [5]:
query = """
MATCH (n:MetaNode) RETURN n;
"""
graph.run(query).to_data_frame()

,n
0,"{'nodeName': 'State', 'synonyms': 'Alternate names of state (string[])', 'name': 'Name of state (string)', 'location': 'Latitude and longitude in WGS-84 format (point{crs:WGS-84})', 'id': 'Geonames.org id for location (string)', 'population': 'Population (int)'}"
1,"{'nodeName': 'City', 'synonyms': 'Alternate names of city (string[])', 'name': 'Name of city (string)', 'location': 'Latitude and longitude in WGS-84 format (point{crs:WGS-84})', 'id': 'Geonames.org id for location (string)', 'population': 'Population (int)'}"
2,"{'nodeName': 'Symptom', 'name': 'Name of symptom (string)', 'id': 'Symptom id from Symptom Ontology (string)'}"
3,"{'nodeName': 'Patient', 'firstName': 'First name (string)', 'lastName': 'Last name (string)', 'smoker': 'Patient is a smoker (boolean)', 'sex': 'Biological sex (string)', 'id': 'Unique patient id (string)', 'age': 'Age (int)'}"
4,"{'nodeName': 'Disease', 'name': 'Name of disease from Human Disease Ontology (string)', 'id': 'Disease id from Human Disease Ontology (string)'}"


### Get number of Nodes

In [6]:
query = """
MATCH (n)
RETURN COUNT(n);
"""
n = graph.evaluate(query)
print(f'Total number of nodes: {n}')

Total number of nodes: 19


### Get number of nodes by node label

In [7]:
query = """
MATCH (n) RETURN labels(n)[0] AS Node, COUNT(n) AS Count
ORDER BY Count DESC
"""
graph.run(query).to_data_frame()

,Node,Count
0,MetaNode,5
1,Symptom,4
2,Disease,3
3,Patient,3
4,City,3
5,State,1


### Where do the patients live?

In [8]:
query = """
MATCH (p:Patient)-[:LIVES_IN]->(c:City)
RETURN p.firstName AS Firstname, p.lastName AS Lastname, c.name AS City;
"""
graph.run(query).to_data_frame()

,Firstname,Lastname,City
0,John,Doe,San Francisco
1,Jane,Doe,Los Angeles
2,James,Bond,San Diego


### Which patients smoke?

In [9]:
query = """
MATCH (p:Patient)
WHERE p.smoker = True
RETURN p.firstName AS Firstname, p.lastName AS Lastname;
"""
graph.run(query).to_data_frame()

,Firstname,Lastname
0,John,Doe
1,James,Bond


## What kind of diseases do the smokers have?

In [10]:
query = """
MATCH (p:Patient)-[:DIAGNOSED_WITH]->(d:Disease)
WHERE p.smoker = True
RETURN p.firstName AS Firstname, p.lastName AS Lastname, d.name as Disease;
"""
graph.run(query).to_data_frame()

,Firstname,Lastname,Disease
0,John,Doe,type 2 diabetes mellitus
1,James,Bond,COVID-19


### Which patients live in California?

In [11]:
query = """
MATCH (p:Patient)-[:LIVES_IN]->(c:City)-[:LOCATED_IN]->(s:State)
WHERE s.name = 'California'
RETURN p.firstName AS Firstname, p.lastName AS Lastname, c.name AS City;
"""
graph.run(query).to_data_frame()

,Firstname,Lastname,City
0,James,Bond,San Diego
1,Jane,Doe,Los Angeles
2,John,Doe,San Francisco


### How many patients live in California?

In [12]:
query = """
MATCH (p:Patient)-[:LIVES_IN]->(:City)-[:LOCATED_IN]->(s:State) 
WHERE s.name = 'California' 
RETURN COUNT(p)
"""
n = graph.evaluate(query)
print(f'Patients in California: {n}')

Patients in California: 3


### Full text query for Diabetes

In [13]:
query = """
CALL db.index.fulltext.queryNodes('fulltext', $keyword) YIELD node, score
RETURN node.id as id, node.name as disease, score
"""

In [14]:
keyword = "Diabetes"
graph.run(query, keyword=keyword).to_data_frame()

,id,disease,score
0,DOID:9352,type 2 diabetes mellitus,0.704895
